#####Objective: Pre/Post Disambig Counts
- Business Owner: Ana Riquezes
- Data Assesment Owner: Anna Soloveva

In [0]:
%sh
curl https://packages.microsoft.com/keys/microsoft.asc | apt-key add -
curl https://packages.microsoft.com/config/ubuntu/16.04/prod.list > /etc/apt/sources.list.d/mssql-release.list
apt-get update
ACCEPT_EULA=Y apt-get install msodbcsql17

% Total % Received % Xferd Average Speed Time Time Time Current
 Dload Upload Total Spent Left Speed

 0 0 0 0 0 0 0 0 --:--:-- --:--:-- --:--:-- 0Warning: apt-key output should not be parsed (stdout is not a terminal)

100 983 100 983 0 0 2678 0 --:--:-- --:--:-- --:--:-- 2678
OK
 % Total % Received % Xferd Average Speed Time Time Time Current
 Dload Upload Total Spent Left Speed

 0 0 0 0 0 0 0 0 --:--:-- --:--:-- --:--:-- 0
 0 0 0 0 0 0 0 0 --:--:-- --:--:-- --:--:-- 0
100 79 100 79 0 0 254 0 --:--:-- --:--:-- --:--:-- 254
Get:1 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Hit:2 https://repos.azul.com/zulu/deb stable InRelease
Get:3 http://security.ubuntu.com/ubuntu focal-security/restricted amd64 Packages [1401 kB]
Hit:4 http://archive.ubuntu.com/ubuntu focal InRelease
Get:5 http://security.ubuntu.com/ubuntu focal-security/main amd64 Packages [2035 kB]
Get:6 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Get:7 http://security.ubuntu.com/ubuntu focal-security/universe amd64 Packages [884 kB]
Hit:8 https://packages.microsoft.com/ubuntu/16.04/prod xenial InRelease
Get:9 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Get:10 http://archive.ubuntu.com/ubuntu focal-updates/restricted amd64 Packages [1513 kB]
Get:11 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 Packages [2483 kB]
Get:12 http://archive.ubuntu.com/ubuntu focal-updates/universe amd64 Packages [1161 kB]
Get:13 http://archive.ubuntu.com/ubuntu focal-updates/multiverse amd64 Packages [30.2 kB]
Get:14 http://archive.ubuntu.com/ubuntu focal-backports/universe amd64 Packages [27.1 kB]
Fetched 9870 kB in 1s (6940 kB/s)
Reading package lists...
Reading package lists...
Building dependency tree...
Reading state information...
msodbcsql17 is already the newest version (17.8.1.1-1).
0 upgraded, 0 newly installed, 0 to remove and 62 not upgraded.

In [0]:
#%sh    
#apt-get -y install unixodbc-dev
#/databricks/python/bin/pip install pyodbc

In [0]:
import pyodbc

conn = pyodbc.connect( 'DRIVER={ODBC Driver 17 for SQL Server};'
                       'SERVER=prod.db.tess-db.h1.co;'
                       'DATABASE=Tesseract;'
                      'UID=anna.soloveva;'  # Input your username
                       'PWD=@H1TdB293#')    # Input your Passward

In [0]:
# Importing appropriate libraries 

from pyspark.sql import *
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)
from pyspark.sql import functions as F
import pandas as pd
import numpy as np
import datetime as dt

In [0]:
timeStamp = dt.datetime.now().strftime('"%m/%d/%YYYY"')
timeStamp = timeStamp[1:-4].replace("/","").replace(":",".").replace(", ","_")
timeStamp

Out[27]: '07212022'

In [0]:
# Pre Disumbig - pp.PublicationID, pub.Title, pub.Abstract, pub.Journal, pub.Keywords, pub.Publisher, pub.DatePublished
# AND (pub.DOI != '' OR pub.PMID != '')

pubs = sql("""SELECT p.FullName, p.PeopleID as PeopleMasterID,  p.PubsCount as PreDisambigPubCounts, spl.publicationCount as PostDisambigPubsCount,  p.PrimarySpecialty, p.SecondarySpecialty, p.Designation, p.NPINumber, p.LinkedInURL, p.TwitterURL, p.Email
FROM tesseract_curie.people_latest p
LEFT JOIN curie.sliced_people_latest spl on p.PeopleID = spl.id
WHERE p.PubsCount is not null""")

In [0]:
pubs = pubs.toPandas().replace(np.nan, '', regex=True)
display(pubs)

FullName,PeopleMasterID,PreDisambigPubCounts,PostDisambigPubsCount,PrimarySpecialty,SecondarySpecialty,Designation,NPINumber,LinkedInURL,TwitterURL,Email
Aboud Affi,1175,1,12,Gastroenterology,,MD,1003891250,https://www.linkedin.com/in/aboud-affi-md-9683691a8,,
Pouria Kashkouli,1202,1,1,,Hospice and Palliative Medicine,MD | MSc,1003892282,,,
Esteban E Golombievski,6856,3,4,Psychiatry & Neurology - Neurology,,MD,1023339033,,,
Shivajirao Prakash Patil,6860,1,27,Family Medicine,,MD,1023339173,,,spatil@vidanthealth.com
Karen Annette Beaty,7424,1,11,Physician Assistant,,,1033222757,,,
Mark S Borer,7543,1,2,Psychiatry & Neurology - Child & Adolescent Psychiatry; Psychiatry & Neurology - Psychiatry,,,1033228077,https://linkedin.com/in/carol-bugglin-69680016,,
Claude Steriade,8955,48,34,Psychiatry & Neurology - Neurology,,CM | MD,1013369446,https://www.linkedin.com/in/claude-steriade-384183111,,
Tamer A Ghanem,10871,48,101,Otolaryngology; Otolaryngology - Plastic Surgery within the Head & Neck,,MD | PhD,1023185394,,,
Michael J Brenner,11276,34,567,Otolaryngology,Otolaryngology - Sleep Medicine; Otolaryngology - Otolaryngology/Facial Plastic Surgery; Otolaryngology - Plastic Surgery within the Head & Neck; Otolaryngology - Facial Plastic Surgery,FACS | MD,1023146776,,,mbren@med.umich.edu|mbren@umich.edu
Thottathil Viswanathan Gopan,11495,1,7,"Endocrinology, Diabetes & Metabolism",,FACP | MD,1023148939,,,


In [0]:
pubs = pubs[pubs.PreDisambigPubCounts != pubs.PostDisambigPubsCount]
display(pubs)

FullName,PeopleMasterID,PreDisambigPubCounts,PostDisambigPubsCount,PrimarySpecialty,SecondarySpecialty,Designation,NPINumber,LinkedInURL,TwitterURL,Email
Aboud Affi,1175,1,12,Gastroenterology,,MD,1003891250,https://www.linkedin.com/in/aboud-affi-md-9683691a8,,
Esteban E Golombievski,6856,3,4,Psychiatry & Neurology - Neurology,,MD,1023339033,,,
Shivajirao Prakash Patil,6860,1,27,Family Medicine,,MD,1023339173,,,spatil@vidanthealth.com
Karen Annette Beaty,7424,1,11,Physician Assistant,,,1033222757,,,
Mark S Borer,7543,1,2,Psychiatry & Neurology - Child & Adolescent Psychiatry; Psychiatry & Neurology - Psychiatry,,,1033228077,https://linkedin.com/in/carol-bugglin-69680016,,
Claude Steriade,8955,48,34,Psychiatry & Neurology - Neurology,,CM | MD,1013369446,https://www.linkedin.com/in/claude-steriade-384183111,,
Tamer A Ghanem,10871,48,101,Otolaryngology; Otolaryngology - Plastic Surgery within the Head & Neck,,MD | PhD,1023185394,,,
Michael J Brenner,11276,34,567,Otolaryngology,Otolaryngology - Sleep Medicine; Otolaryngology - Otolaryngology/Facial Plastic Surgery; Otolaryngology - Plastic Surgery within the Head & Neck; Otolaryngology - Facial Plastic Surgery,FACS | MD,1023146776,,,mbren@med.umich.edu|mbren@umich.edu
Thottathil Viswanathan Gopan,11495,1,7,"Endocrinology, Diabetes & Metabolism",,FACP | MD,1023148939,,,
Bharath Srivatsa,11619,2,4,Pediatrics - Neonatal-Perinatal Medicine,,MD,1003814112,,,Bharath_Srivatsa@mednax.com


In [0]:
pubs['Seed_Prct'] = (pubs.PreDisambigPubCounts / pubs.PostDisambigPubsCount)
pubs

Out[31]:

,FullName,PeopleMasterID,PreDisambigPubCounts,PostDisambigPubsCount,PrimarySpecialty,SecondarySpecialty,Designation,NPINumber,LinkedInURL,TwitterURL,Email,Seed_Prct
0,Aboud Affi,1175,1,12,Gastroenterology,,MD,1003891250,https://www.linkedin.com/in/aboud-affi-md-9683...,,,0.083333
2,Esteban E Golombievski,6856,3,4,Psychiatry & Neurology - Neurology,,MD,1023339033,,,,0.750000
3,Shivajirao Prakash Patil,6860,1,27,Family Medicine,,MD,1023339173,,,spatil@vidanthealth.com,0.037037
4,Karen Annette Beaty,7424,1,11,Physician Assistant,,,1033222757,,,,0.090909
5,Mark S Borer,7543,1,2,Psychiatry & Neurology - Child & Adolescent Ps...,,,1033228077,https://linkedin.com/in/carol-bugglin-69680016,,,0.500000
...,...,...,...,...,...,...,...,...,...,...,...,...
914948,Julien Taieb,12028723,1,481,,,,,,,julien.taieb@aphp.fr,0.002079
914949,Laure Seugé,12028767,2,5,,,,,https://www.linkedin.com/in/laure-seug%C3%A9-6...,,,0.400000
914950,Marc Pocard,12028847,1,448,,,,,,,marc.pocard@lrb.aphp.fr,0.002232
914951,Marco Antonio Barajas Romero,12028851,2,1,,,,,,,,2.000000


In [0]:
pubs_filtered = pubs.loc[pubs['Seed_Prct'] <0.02]
len(pubs_filtered)

Out[32]: 54211

In [0]:
query = f"""SELECT pp.PeopleMasterID, pp.listingURL as ListingURL, pp.personListingURL as ProfileURL, tag.Tier
FROM Tesseract.gpov1.Person_Premaster pp  WITH(NOLOCK)
LEFT JOIN (SELECT distinct pp.ParentID, ppd.projectpersontagid as Tier
FROM Tesseract.pm.ProjectPerson pp WITH (NOLOCK)
LEFT JOIN Tesseract.pm.ProjectPersonTagDetail ppd WITH (NOLOCK) ON ppd.parentid = pp.projectpersonid
WHERE ppd.projectpersontagid = 10 OR ppd.projectpersontagid = 11 OR ppd.projectpersontagid = 17 )tag
ON tag.ParentID = pp.PeopleMasterID
"""

url_tag = pd.read_sql(query,conn)
display(url_tag)

PeopleMasterID,ListingURL,ProfileURL,Tier
1.0027065E7,null,null,17.0
575188.0,https://doctors.nyp.org/?aff=false,https://doctors.nyp.org/john-c-markowitz-md/40-east-83rd-street|https://www.columbiapsychiatry.org/profile/john-c-markowitz-md#areas-of-expertise-conditions-treated,10.0
2691354.0,null,null,11.0
488746.0,null,https://pubmed.ncbi.nlm.nih.gov/35230440|https://vivo.weill.cornell.edu/display/cwid-dss2001|https://www.nyp.org/search-results?search=Douglas%20S.%20Scherr,10.0
5490322.0,null,null,11.0
2894207.0,null,null,10.0
2171923.0,null,null,17.0
697714.0,null,null,17.0
2256639.0,null,null,10.0
3503898.0,null,https://doctors.bjc.org/wlp2/bjc/doctors/info/AKT0052R/Robert-T-Naismith-MD|https://icts.wustl.edu/people/robert-t-naismith-md,10.0


In [0]:
# Merge 
pubs_filtered = pd.merge(pubs_filtered, url_tag, on ='PeopleMasterID', how = 'inner').sort_values(by = 'Seed_Prct')
pubs_filtered = pubs_filtered.drop_duplicates()

In [0]:
pubs_filtered['Tags'] = pubs_filtered['Tier'].astype(str)
pubs_filtered = pubs_filtered.drop(columns = ['Tier'])

pubs_filtered['ProfileURLs'] = pubs_filtered['ProfileURL'].astype(str)
pubs_filtered = pubs_filtered.drop(columns = ['ProfileURL'])

pubs_filtered['ListingURLs'] = pubs_filtered['ListingURL'].astype(str)
pubs_filtered = pubs_filtered.drop(columns = ['ListingURL'])

In [0]:
# Perform a groupby and apply a string join.
tags1 = pubs_filtered[["PeopleMasterID", "Tags"]].drop_duplicates(["PeopleMasterID", "Tags"])
tags1 = tags1[tags1.Tags != 'nan']
tags1 = tags1.groupby('PeopleMasterID')['Tags'].apply(' | '.join).reset_index()

# Perform a groupby and apply a string join.
pr_url = pubs_filtered[["PeopleMasterID", "ProfileURLs"]].drop_duplicates(["PeopleMasterID", "ProfileURLs"])
pr_url = pr_url[pr_url.ProfileURLs != 'None']
pr_url = pr_url.groupby('PeopleMasterID')['ProfileURLs'].apply(' | '.join).reset_index()

# Perform a groupby and apply a string join.
lis_url = pubs_filtered[["PeopleMasterID", "ListingURLs"]].drop_duplicates(["PeopleMasterID", "ListingURLs"])
lis_url = lis_url[lis_url.ListingURLs != 'None']
lis_url = lis_url.groupby('PeopleMasterID')['ListingURLs'].apply(' | '.join).reset_index()

In [0]:
pubs_filtered = pd.merge(pubs_filtered, pr_url, on = 'PeopleMasterID' , how = 'left').sort_values(by = 'PeopleMasterID')
pubs_filtered = pubs_filtered.drop(columns = ['ProfileURLs_x'])
pubs_filtered = pubs_filtered.rename(columns = {'ProfileURLs_y':'ProfileURLs'})
pubs_filtered = pubs_filtered.drop_duplicates()

In [0]:
pubs_filtered = pd.merge(pubs_filtered, lis_url, on = 'PeopleMasterID' , how = 'left').sort_values(by = 'PeopleMasterID')
pubs_filtered = pubs_filtered.drop(columns = ['ListingURLs_x'])
pubs_filtered = pubs_filtered.rename(columns = {'ListingURLs_y':'ListingURLs'})
pubs_filtered = pubs_filtered.drop_duplicates()

In [0]:
pubs_filtered = pd.merge(pubs_filtered, tags1, on = 'PeopleMasterID' , how = 'left').sort_values(by = 'PeopleMasterID')
pubs_filtered = pubs_filtered.drop(columns = ['Tags_x'])
pubs_filtered = pubs_filtered.rename(columns = {'Tags_y':'Tags'})
pubs_filtered = pubs_filtered.drop_duplicates()
#display(pubs_filtered)

/databricks/spark/python/pyspark/sql/pandas/conversion.py:300: UserWarning: createDataFrame attempted Arrow optimization because 'spark.sql.execution.arrow.pyspark.enabled' is set to true; however, failed by the reason below:
 Duplicate column names found: ['FullName', 'PeopleMasterID', 'PreDisambigPubCounts', 'PostDisambigPubsCount', 'PrimarySpecialty', 'SecondarySpecialty', 'Designation', 'NPINumber', 'LinkedInURL', 'TwitterURL', 'Email', 'Seed_Prct', 'ProfileURLs', 'ListingURLs', 'Tags', 'Tags']
Attempting non-optimization as 'spark.sql.execution.arrow.pyspark.fallback.enabled' is set to true.
 warnings.warn(msg)

--------------------------------------------------------------------------- 
 ValueError Traceback (most recent call last)
 <command-2677850810126803> in <module> 
 3 pubs_filtered = pubs_filtered . rename ( columns = { 'Tags_y' : 'Tags' } ) 
 4 pubs_filtered = pubs_filtered . drop_duplicates ( ) 
 ----> 5 display ( pubs_filtered ) 

 /databricks/python_shell/scripts/PythonShellImpl.py in display (self, input, *args, **kwargs) 
 1038 if self . sparkSession is None : 
 1039 raise Exception ( 'SparkSession is required for display(pandas.DataFrame).' ) 
 -> 1040 self . display ( self . sparkSession . createDataFrame ( input ) ) 
 1041 elif type ( input ) . __module__ in ( 'databricks.koalas.frame' , 'pyspark.pandas.frame' ) and \ 
 1042 type ( input ) . __name__ == 'DataFrame' : 

 /databricks/spark/python/pyspark/sql/session.py in createDataFrame (self, data, schema, samplingRatio, verifySchema) 
 718 if has_pandas and isinstance ( data , pandas . DataFrame ) : 
 719 # Create a DataFrame from pandas DataFrame. 
 --> 720 return super(SparkSession, self).createDataFrame(
 721 data, schema, samplingRatio, verifySchema)
 722 return self . _create_dataframe ( data , schema , samplingRatio , verifySchema ) 

 /databricks/spark/python/pyspark/sql/pandas/conversion.py in createDataFrame (self, data, schema, samplingRatio, verifySchema) 
 308 warnings . warn ( msg ) 
 309 raise 
 --> 310 data = self . _convert_from_pandas ( data , schema , timezone ) 
 311 return self . _create_dataframe ( data , schema , samplingRatio , verifySchema ) 
 312 

 /databricks/spark/python/pyspark/sql/pandas/conversion.py in _convert_from_pandas (self, pdf, schema, timezone) 
 351 
 352 # Convert pandas.DataFrame to list of numpy records 
 --> 353 np_records = pdf . to_records ( index = False ) 
 354 
 355 # Check if any columns need to be fixed for Spark to infer properly 

 /databricks/python/lib/python3.8/site-packages/pandas/core/frame.py in to_records (self, index, column_dtypes, index_dtypes) 
 2062 raise ValueError ( msg ) 
 2063 
 -> 2064 return np . rec . fromarrays ( arrays , dtype = { "names" : names , "formats" : formats } ) 
 2065 
 2066 @ classmethod

 /databricks/python/lib/python3.8/site-packages/numpy/core/records.py in fromarrays (arrayList, dtype, shape, formats, names, titles, aligned, byteorder) 
 658 
 659 if dtype is not None : 
 --> 660 descr = sb . dtype ( dtype ) 
 661 else : 
 662 descr = format_parser ( formats , names , titles , aligned , byteorder ) . dtype

 ValueError : name already used as a name or title

In [0]:
len(pubs_filtered)

Out[44]: 49438

#### Download Report

In [0]:
pubs_filtered.to_csv(f"/dbfs/FileStore/tables/CQA_1006_Pre_Post_Disambig_publications.csv", index = False)
print(f'https://dbc-593c2c2d-f6be.cloud.databricks.com/files/tables/CQA_1006_Pre_Post_Disambig_publications.csv?o=960084232686341')

https://dbc-593c2c2d-f6be.cloud.databricks.com/files/tables/CQA_1006_Pre_Post_Disambig_publications.csv?o=960084232686341